######Google Drive, import libraries and data loading

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
%cd /gdrive/MyDrive/Challenge2/

/gdrive/MyDrive/Challenge2


In [ ]:
#!unzip training_dataset_homework2

Archive:  training_dataset_homework2.zip
  inflating: y_train.npy             
  inflating: x_train.npy             


In [ ]:
import tensorflow as tf
import numpy as np
import os
import random
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
plt.rc('font', size=16) 
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import MinMaxScaler
import warnings
import logging
from sklearn.model_selection import train_test_split

tfk = tf.keras
tfkl = tf.keras.layers

# Random seed for reproducibility
seed = 127

random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

In [ ]:
X = np.load("x_train.npy")
y = np.load("y_train.npy")

In [ ]:
#Split into train-validation (our test) set
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=seed)

In [ ]:
from sklearn.utils import class_weight
cws = class_weight.compute_class_weight(class_weight = "balanced" , classes = np.unique(y_train), y= y_train)

In [ ]:
# Convert the sparse labels to categorical values
y_train = tfk.utils.to_categorical(y_train)
y_val = tfk.utils.to_categorical(y_val)

####Training and Ensemble Model

In [ ]:
input_shape = X.shape[1:]
classes = y_train.shape[-1]
batch_size = 32 #64
epochs = 200

#####LSTM

In [ ]:
def build_LSTM_classifier(input_shape, classes):
    # Build the neural network layer by layer
    input_layer = tfkl.Input(shape=input_shape, name='Input')

    # Feature extractor
    lstm = tfkl.LSTM(128,return_sequences=True)(input_layer)
    lstm = tfkl.LSTM(128)(lstm)
    dropout = tfkl.Dropout(.2, seed=seed)(lstm)

    # Classifier

    classifier = tfkl.Dense(128, activation='relu')(dropout)
    dropout = tfkl.Dropout(.2, seed=seed)(classifier)
    classifier = tfkl.Dense(128, activation='relu')(dropout)
    output_layer = tfkl.Dense(classes, activation='softmax')(classifier)

    # Connect input and output through the Model class
    model = tfk.Model(inputs=input_layer, outputs=output_layer, name='model')

    # Compile the model
    model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(), metrics='accuracy')

    # Return the model
    return model

In [ ]:
lstm = build_LSTM_classifier(input_shape, classes)
lstm.summary()

#####Convolutions

In [ ]:
def build_1DCNN_classifier(input_shape, classes):
    # Build the neural network layer by layer
    input_layer = tfkl.Input(shape=input_shape, name='Input')

    # Feature extractor
    cnn = tfkl.Conv1D(128,3,padding='same',activation='relu')(input_layer)
    cnn = tfkl.AveragePooling1D()(cnn)# 
    cnn = tfkl.Conv1D(256,3,padding='valid',activation='relu')(cnn)
    gap = tfkl.GlobalAveragePooling1D()(cnn)
    dropout = tfkl.Dropout(.3, seed=seed)(gap)

    # Classifier
    classifier = tfkl.Dense(128, activation='relu')(dropout)
    dropout = tfkl.Dropout(.3, seed=seed)(classifier)
    classifier = tfkl.Dense(128, activation='relu')(dropout)
    output_layer = tfkl.Dense(classes, activation='softmax')(classifier)

    # Connect input and output through the Model class
    model = tfk.Model(inputs=input_layer, outputs=output_layer, name='model')

    # Compile the model
    model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(), metrics='accuracy')

    # Return the model
    return model

In [ ]:
cnn1 = build_1DCNN_classifier(input_shape, classes)
cnn1.summary()

In [ ]:
def build_1DCNN_classifier2(input_shape, classes):
    # Build the neural network layer by layer
    input_layer = tfkl.Input(shape=input_shape, name='Input')

    # Feature extractor
    cnn = tfkl.Conv1D(16,3,padding='same',activation='relu')(input_layer)
    cnn = tfkl.Conv1D(32,3,padding='same',activation='relu')(cnn)
    cnn = tfkl.Conv1D(64,3,padding='same',activation='relu')(cnn)

    gap = tfkl.GlobalAveragePooling1D()(cnn)
    dropout = tfkl.Dropout(.3, seed=seed)(gap)

    # Classifier
    classifier = tfkl.Dense(128, activation='relu')(dropout)
    dropout = tfkl.Dropout(.3, seed=seed)(classifier)
    classifier = tfkl.Dense(128, activation='relu')(dropout)
    output_layer = tfkl.Dense(classes, activation='softmax')(classifier)

    # Connect input and output through the Model class
    model = tfk.Model(inputs=input_layer, outputs=output_layer, name='model')

    # Compile the model
    model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(), metrics='accuracy')

    # Return the model
    return model

In [ ]:
cnn2 = build_1DCNN_classifier2(input_shape, classes)
cnn2.summary()

In [ ]:
def build_1DCNN_classifier3(input_shape, classes):
    # Build the neural network layer by layer
    input_layer = tfkl.Input(shape=input_shape, name='Input')

    # Feature extractor
    cnn = tfkl.Conv1D(128,3,padding='same',activation='relu')(input_layer)
    cnn = tfkl.Conv1D(256,3,dilation_rate = 2,padding='same',activation='relu')(cnn)
    gap = tfkl.GlobalAveragePooling1D()(cnn)
    dropout = tfkl.Dropout(.3, seed=seed)(gap)

    # Classifier
    classifier = tfkl.Dense(128, activation='relu')(dropout)
    dropout = tfkl.Dropout(.3, seed=seed)(classifier)
    classifier = tfkl.Dense(128, activation='relu')(dropout)
    output_layer = tfkl.Dense(classes, activation='softmax')(classifier)

    # Connect input and output through the Model class
    model = tfk.Model(inputs=input_layer, outputs=output_layer, name='model')

    # Compile the model
    model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(), metrics='accuracy')

    # Return the model
    return model

In [ ]:
cnn3 = build_1DCNN_classifier3(input_shape, classes)
cnn3.summary()

#####ConvLstm

In [ ]:
def build_CONV_LSTM_model(input_shape, classes):
    # Build the neural network layer by layer
    input_layer = tfkl.Input(shape=input_shape, name='Input')

    convlstm = tfkl.Conv1D(32, 3, padding='same', activation='relu')(input_layer)
    convlstm = tfkl.Conv1D(64, 3, padding='same', activation='relu')(convlstm)
    convlstm = tfkl.Conv1D(128, 3, padding='same', activation='relu')(convlstm)

    convlstm = tfkl.Bidirectional(tfkl.LSTM(128, return_sequences=True))(convlstm)
    convlstm = tfkl.Bidirectional(tfkl.LSTM(128))(convlstm)

    convlstm = tfkl.Dropout(.2)(convlstm)

    convlstm = tfkl.Dense(64, activation='relu')(convlstm)

    convlstm = tfkl.Dropout(.2, seed=seed)(convlstm)

    convlstm = tfkl.Dense(64, activation='relu')(convlstm)

    output_layer = tfkl.Dense(classes, activation='softmax')(convlstm)

    # Connect input and output through the Model class
    model = tfk.Model(inputs=input_layer, outputs=output_layer, name='model')

    # Compile the model
    model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(), metrics='accuracy')

    # Return the model
    return model    


In [ ]:
convlstm = build_CONV_LSTM_model(input_shape=input_shape, classes = 12)
convlstm.summary()

#####Training

In [ ]:
# Train the model
model = cnn1 #name of the model to train
history = model.fit(
    x = X_train,
    y = y_train,
    batch_size = batch_size,
    epochs = 300,
    validation_data= (X_val,y_val),
    callbacks = [
        tfk.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', patience=70, restore_best_weights=True),
        tfk.callbacks.ReduceLROnPlateau(monitor='val_accuracy', mode='max', patience=30, factor=0.5, min_lr=1e-5)
    ]
).history

#####Ensemble Model

In [ ]:
#final_model = tfk.models.load_model("FinalSubmission")

In [ ]:
cnn1._name = "cnn1"
cnn2._name = "cnn2"
cnn3._name = "cnn3"
lstm._name = 'lstm'
convlstm._name = 'convlstm'

In [ ]:
models = []

models.append(cnn1)
models.append(cnn2)
models.append(cnn3)
models.append(convlstm)
models.append(lstm)

In [ ]:
def ensembleModels(models, model_input):
    # collect outputs of models in a list
    yModels=[model(model_input) for model in models] 
    # averaging outputs
    yAvg=tfk.layers.average(yModels) 
    # build model from same input and avg output
    modelEns = tfk.Model(inputs=model_input, outputs=yAvg,    name='ensemble')  
   
    return modelEns

In [ ]:
model_input = tfk.Input(shape=models[0].input_shape[1:]) 
modelEns = ensembleModels(models,model_input)
modelEns.compile(optimizer=tfk.optimizers.Adam(),
              loss=tfk.losses.CategoricalCrossentropy(),
              metrics='accuracy')
modelEns.summary()

In [ ]:
# modelEns.save("3cnn+lstm+convlstm")